In [1]:
!mkdir bin_data

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets tiktoken wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [26]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset


! pip install tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import numpy as np
import os
import tiktoken
import gc

In [5]:
num_proc = 8

if __name__ == '__main__':


    cache_directory = "/content/dataset"

    loaded_dataset = load_dataset("Skylion007/openwebtext", num_proc=8, cache_dir=cache_directory)
    print(f"Dataset information for the original set with the number of documents: {loaded_dataset}")


    train_test_split_data = loaded_dataset["train"].train_test_split(test_size=0.45, seed=2357, shuffle=True)
    _ = train_test_split_data.pop('test')

    print(f"Dataset after randomly eliminating 30% of the data: {train_test_split_data}")

    train_test_split_data = train_test_split_data["train"].train_test_split(test_size=0.001, seed=2357, shuffle=True)
    train_test_split_data['val'] = train_test_split_data.pop('test')

    print(f"Final dataset with validation set: {train_test_split_data}")

    text_encoder = tiktoken.get_encoding("gpt2")

    def process_example(example):
        encoded_ids = text_encoder.encode_ordinary(example['text']) + [text_encoder.eot_token]
        result = {'ids': encoded_ids, 'len': len(encoded_ids)}
        return result

    processed_data = train_test_split_data.map(process_example, remove_columns=['text'], desc="Tokenization in progress", num_proc=num_proc)

    for data_split, dataset in processed_data.items():


        lengths = np.array(dataset['len'])
        array_length = np.uint64(lengths.sum())

        file_path = "/content/bin_data/{}.bin".format(data_split)
        data_type = np.dtype(np.uint16)

        memory_mapped_array = np.memmap(file_path,
                                        dtype=data_type,
                                        mode='w+',
                                        shape=(array_length,))


        index = 0
        for batch_index in tqdm(range(1024), desc=f'writing {file_path}'):

            batch_array = np.concatenate(dataset.shard(num_shards=1024, index=batch_index, contiguous=True).with_format('numpy')['ids'])

            slice_start = index
            slice_end = index + len(batch_array)

            copied_array = np.copy(batch_array)

            memory_mapped_array[slice_start:slice_end] = copied_array


            index = index +  len(copied_array)

        memory_mapped_array.flush()


Original Dataset info with no of docs:  DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8013769
    })
})
After randomly removing 30% data:  DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4407572
    })
})
Final dataset with val:  DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4403164
    })
    val: Dataset({
        features: ['text'],
        num_rows: 4408
    })
})


tokenizing the splits (num_proc=8):   0%|          | 0/4403164 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/4408 [00:00<?, ? examples/s]

writing /content/bin_data/val.bin: 100%|██████████| 1024/1024 [00:01<00:00, 537.42it/s]


In [6]:
# hyperparameters
block_size = 256
batch_size = 16
learning_rate = 3e-4

eval_iters = 10
n_head = 6
n_embd = 512
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)

device_type = 'cuda' if 'cuda' in device else 'cpu'

device_type

'cuda'

In [9]:
gpt_encoding = tiktoken.get_encoding("gpt2")
gpt_encoding_attrs = dir(gpt_encoding)
print(gpt_encoding_attrs)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_core_bpe', '_encode_bytes', '_encode_only_native_bpe', '_encode_single_piece', '_mergeable_ranks', '_pat_str', '_special_tokens', 'decode', 'decode_batch', 'decode_bytes', 'decode_bytes_batch', 'decode_single_token_bytes', 'decode_tokens_bytes', 'decode_with_offsets', 'encode', 'encode_batch', 'encode_ordinary', 'encode_ordinary_batch', 'encode_single_token', 'encode_with_unstable', 'eot_token', 'max_token_value', 'n_vocab', 'name', 'special_tokens_set', 'token_byte_values']


In [14]:
vocab_size = gpt_encoding.n_vocab

In [15]:
directory_path = "/content/bin_data"

train_data_path = os.path.join(directory_path, 'train.bin')
train_data = np.memmap(train_data_path, dtype=np.uint16, mode='r')[:100000]

val_data_path = os.path.join(directory_path, 'val.bin')
val_data = np.memmap(val_data_path, dtype=np.uint16, mode='r')

In [17]:
def get_batch(split):

    dataset = train_data if split == 'train' else val_data

    dataset_length = len(dataset)

    upper_limit = dataset_length - block_size

    indices = torch.randint(upper_limit, (batch_size,))

    input_subarrays = [dataset[i:i+block_size] for i in indices]
    target_subarrays = [dataset[i+1:i+1+block_size] for i in indices]

    input_tensors = [torch.from_numpy(subarray.astype(np.int64)) for subarray in input_subarrays]
    target_tensors = [torch.from_numpy(subarray.astype(np.int64)) for subarray in target_subarrays]

    input_data = torch.stack(input_tensors)
    target_data = torch.stack(target_tensors)

    if device_type == 'cuda':

        input_data = input_data.pin_memory()
        target_data = target_data.pin_memory()


        input_data = input_data.to(device, non_blocking=True)
        target_data = target_data.to(device, non_blocking=True)
    else:

        input_data = input_data.to(device)
        target_data = target_data.to(device)


    return input_data, target_data


In [21]:
@torch.no_grad()

def estimate_loss():

    result = {}

    model.eval()

    for dataset_type in ['train', 'val']:

        losses_list = torch.zeros(shape=(eval_iters,), dtype=torch.float32)

        for iteration in range(eval_iters):

            input_data = get_batch(dataset_type)[0]
            target_data = get_batch(dataset_type)[1]

            predictions, current_loss = model(input_data, target_data)

            losses_list[iteration] = model(input_data, target_data)[1].item()

        mean_loss = sum(losses_list) / len(losses_list)
        result[dataset_type] = mean_loss


    model.train()

    return result


In [ ]:
estimate_loss()

{'train': tensor(10.9383), 'val': tensor(10.9433)}

In [23]:
class Head(nn.Module):

    def __init__(self, head_size):

        super().__init__()

        self.linear_layers = nn.ModuleList([

            nn.Linear(n_embd, head_size, bias=False) for _ in range(3)
        ])

        self.key_layer, self.query_layer, self.value_layer = self.linear_layers

        identity_matrix = torch.eye(block_size)
        lower_triangular_mask = torch.tril(identity_matrix)
        self.register_buffer('tril', lower_triangular_mask)

        self.drop = nn.Dropout(dropout)



    def forward(self, x):

        batch_size = x.shape[0]
        time_steps = x.shape[1]
        channels =   x.shape[2]

        key, query = self.key_layer(x), self.query_layer(x)

        key_transposed = key.transpose(-2, -1)

        attention_scores = torch.matmul(query, key_transposed) / torch.sqrt(torch.tensor(key.shape[-1], dtype=torch.float32))


        attention_mask = torch.where(self.tril[:time_steps, :time_steps] == 0, float('-inf'), attention_scores)

        attention_probs = self.drop(torch.nn.functional.log_softmax(attention_mask, dim=-1).exp())

        value = self.value_layer(x)

        output = attention_probs @ value

        return output


In [24]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()

        self.heads = nn.ModuleList()
        for _ in range(num_heads):
            self.heads.append(Head(head_size))

        self.projection =  nn.Linear(head_size * num_heads, n_embd)

        self.drop = nn.Dropout(dropout)

    def forward(self, x):

        individual_heads = []
        for head_module in self.heads:
            result_of_head = head_module(x)
            individual_heads.append(result_of_head)

        concatenated_heads_expanded = torch.cat(individual_heads, dim=-1)

        projected_out = self.projection(concatenated_heads_expanded)
        dropped_out = self.drop(projected_out)
        return dropped_out

class FeedForward(nn.Module):

    def __init__(self, n_embd, dropout):

        super(FeedForward, self).__init__()
        self.layer1 = nn.Linear(n_embd, 4 * n_embd)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(4 * n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, n_embd, n_head):

        super().__init__()

        self.self_attention = MultiHeadAttention(n_head, (n_embd // n_head))
        self.feed_forward = FeedForward(n_embd,dropout)
        self.linearlayer1, self.linearlayer2 = nn.LayerNorm(n_embd), nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.self_attention(self.linearlayer1(x)) + self.feed_forward(self.linearlayer2(x))
        return x


class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.token_emb_tbl = nn.Embedding(vocab_size, n_embd)
        self.pos_emb_tbl = nn.Embedding(block_size, n_embd)

        attention_blocks = []
        for _ in range(n_layer):
            attention_blocks.append(Block(n_embd, n_head=n_head))
        self.attention_blocks = nn.Sequential(*attention_blocks)

        self.layer_norm_final = nn.LayerNorm(n_embd)
        self.mode_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            torch.nn.init.zeros_(module.bias) if hasattr(module, 'bias') and module.bias is not None else None


    def forward(self, idx, targets=None):

        embedding_indices = idx
        token_embeddings = self.token_emb_tbl(embedding_indices)

        pos_indices = torch.arange(idx.shape[1], device=device)
        positional_embeddings = self.pos_emb_tbl(pos_indices).unsqueeze(0)

        normalized_embeddings = self.layer_norm_final(self.attention_blocks(token_embeddings + positional_embeddings.expand_as(token_embeddings)))

        logits = self.mode_head(normalized_embeddings)

        loss = F.cross_entropy(logits.flatten(0, 1), targets.flatten()) if targets is not None else None

        return logits, loss



    def generate(self, idx, max_new_tokens):

            for _ in range(max_new_tokens):

                currentlogits, _ = self.forward(idx[:, -block_size:])
                currentlogits = currentlogits[:, -1, :]

                nexttoken = torch.multinomial( F.softmax(currentlogits, dim=-1), num_samples=1).view(-1, 1)

                idx = torch.hstack((idx, nexttoken))

            return idx




In [ ]:
def generate_tokens_greedily(initial_indices, newtokens):
  for i in range(newtokens):
        indices_cond = initial_indices[:, -block_size:]
        with torch.no_grad():
          output_logits, loss = model(indices_cond)
        output_logits = output_logits[:, -1, :]
        prob = torch.nn.functional.softmax(output_logits, dim=-1)
        max_indices = torch.max(prob, dim=-1, keepdim=True)[1]
        initial_indices = torch.cat((initial_indices, max_indices), dim=1)
  return initial_indices

In [25]:
model = GPTLanguageModel().to(device)
total_parameters = sum(p.numel() for p in model.parameters())
total_parameters_in_millions = total_parameters / 1e6

In [35]:
model.load_state_dict(torch.load("/content/model_9000_5.3693.pth")["model_state_dict"])


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

optimizer.load_state_dict(torch.load("/content/model_9000_5.3693.pth")["optimizer_state_dict"])

In [ ]:
import wandb
wandb.login(key="bc23ae38b83159616a86ecc84dc25bc2e82d4da4") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "HW5_test", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = 'j1pmsgdm',### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw5-ablations", ### Project should be created in your wandb account
)

wandb: Currently logged in as: sdhote. Use `wandb login --relogin` to force relogin


In [36]:
def generate_and_display_sample(max_tokens=20):

    input_content = ["This chocolate cake ", "The government is doing"]

    encoded_content = gpt_encoding.encode_batch(input_content)
    content_tensors = torch.tensor(encoded_content)
    content_tensors = content_tensors.to(device)

    contents_decoded = gpt_encoding.decode_batch(model.generate(content_tensors, max_tokens).cpu().numpy())

    print("\n####\n".join(contents_decoded))

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

interval_to_save = 1500

dir_to_save= "/content/drive/MyDrive/IDL_HW5/HW5/finetuning_trials/checkpoints"


In [37]:
maximum_iteration = 700000
evaluation_interval = 100


In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 7.7226, val loss 7.6798
The shop offers a  itcephal whether Americans stabbed disorderlyU sets one SERVICES tries poorly onrences change protecting tools power investment and
####
Further investigations revealed that  RSS wouldn greatoni result conquered not.о� this what ^ache
os Republican noted cherish to rumored


  0%|          | 100/700000 [01:03<114:19:21,  1.70it/s]

step 100: train loss 6.8200, val loss 6.7081


  0%|          | 200/700000 [02:05<115:43:24,  1.68it/s]

...

step 72000: train loss 6.4304, val loss 4.7348
The shop offers a я л�ョ���i. Mishler Park, you? Once the morning
####
Further investigations revealed that яв Ｅِي وَث幅由о им


 10%|█         | 72100/700000 [12:15:58<101:12:05,  1.72it/s]

step 72100: train loss 6.5985, val loss 4.7908


 10%|█         | 72195/700000 [12:16:56<100:48:50,  1.73it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.5451, val loss 4.7588
The shop offers a iba”
T
94.
Note: in sync rendezvous took restructure.

####
Further investigations revealed that чих) NASA conducted by a special-funded gives them to snag few years,


  0%|          | 100/700000 [01:04<119:14:33,  1.63it/s]

step 100: train loss 6.5043, val loss 4.7430


  0%|          | 200/700000 [02:06<114:37:51,  1.70it/s]

...

step 70600: train loss 6.3493, val loss 4.7158


 10%|█         | 70700/700000 [11:59:21<101:39:14,  1.72it/s]

step 70700: train loss 6.6546, val loss 4.7131


 10%|█         | 70715/700000 [11:59:33<106:43:14,  1.64it/s]


KeyboardInterrupt: ignored

In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.4455, val loss 4.7327
The shop offers a ройдо лчекит. "S. It is

####
Further investigations revealed that свстоль жласеня таст


  0%|          | 100/700000 [01:04<113:53:54,  1.71it/s]

step 100: train loss 6.4763, val loss 4.7413


  0%|          | 200/700000 [02:05<115:43:11,  1.68it/s]

...

step 72000: train loss 6.5265, val loss 4.7066
The shop offers a уВ health services consumed in the same thing made the Texans and pass.onion http://
####
Further investigations revealed that ю�万َف�ыб用裸Е quantum mechanics


 10%|█         | 72100/700000 [12:13:14<100:48:48,  1.73it/s]

step 72100: train loss 6.5162, val loss 4.7170


 10%|█         | 72196/700000 [12:14:12<101:00:03,  1.73it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
avg = 0
sum = 0
for param_group in optimizer.param_groups:
    avg = avg + param_group['lr']
    sum = sum+ 1
lr = avg/sum
# print("Learning rate: ",lr)

0.0003


In [ ]:
for param_group in optimizer.param_groups:
  param_group['lr'] = 0.0001

In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.5786, val loss 4.7372
The shop offers a ive gear, even what that golf courses in the Cup.
Here's release from both attitudes toward
####
Further investigations revealed that ров有�ы全都аетою有


  0%|          | 100/700000 [01:05<112:58:15,  1.72it/s]

step 100: train loss 6.6316, val loss 4.7105


  0%|          | 200/700000 [02:06<113:26:49,  1.71it/s]

...

step 72000: train loss 6.7127, val loss 4.6156
The shop offers a ять кагутора?


"It may
####
Further investigations revealed that ikka, for we know some point guard Josh McGuinness by my doctors who came to release is


 10%|█         | 72100/700000 [12:17:08<115:32:42,  1.51it/s]

step 72100: train loss 6.6801, val loss 4.6184


 10%|█         | 72196/700000 [12:18:07<102:48:35,  1.70it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.6581, val loss 4.6409
The shop offers a لَهَا وَبَنُ 2, you going northeast corner of the
####
Further investigations revealed that ше нее о jsoucos THEREFORE, Occupy Oakland, so cute


  0%|          | 100/700000 [01:05<121:51:23,  1.60it/s]

step 100: train loss 6.6510, val loss 4.6733


  0%|          | 200/700000 [02:07<115:40:48,  1.68it/s]

...

step 72000: train loss 6.7152, val loss 4.6164
The shop offers a ipsum in an excellent way through this year.


"So, musings so."
####
Further investigations revealed that стама, then-Huff tells deputies that people’sigh Seriously,


 10%|█         | 72100/700000 [12:17:01<101:31:40,  1.72it/s]

step 72100: train loss 6.6389, val loss 4.6942


 10%|█         | 72196/700000 [12:17:59<101:41:18,  1.71it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
for iter in tqdm(range(maximum_iteration)):

    if iter % evaluation_interval == 0 or iter == maximum_iteration - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        wandb.log({'train_loss': losses["train"],'valid_loss': losses["val"]})

    if iter % interval_to_save == 0 or iter == maximum_iteration - 1:
        model.eval()
        generate_and_display_sample()
        model.train()
        torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, os.path.join(dir_to_save,f"model_{losses[False]:.4f}.pth"))
        torch.save({'model_state_dict'         : model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()},  os.path.join(dir_to_save,f"model_{losses['val']:.4f}.pth"))

    xb, yb = get_batch(True)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    del xb, yb, logits
    gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.7402, val loss 4.6577
The shop offers a сиз種�n.
The Massive spikes encodes [Murry (@
####
Further investigations revealed that те вут то что 20 seconds.





  0%|          | 100/700000 [01:04<114:15:10,  1.70it/s]

step 100: train loss 6.7474, val loss 4.6595


  0%|          | 200/700000 [02:06<114:49:00,  1.69it/s]

...

step 72000: train loss 6.7077, val loss 4.6933
The shop offers a ся шやискомыра ж免上
####
Further investigations revealed that agically, facto spokesperson for.
Recently faculty or had LeBron was false to 2000, are all the


 10%|█         | 72100/700000 [12:16:46<100:55:53,  1.73it/s]

step 72100: train loss 6.7296, val loss 4.6861


 10%|█         | 72196/700000 [12:17:45<101:17:33,  1.72it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
for param_group in optimizer.param_groups:
  param_group['lr'] = 0.0001

In [ ]:
for iteration in tqdm(range(maximum_iteration)):

    if iteration % evaluation_interval == 0 or iteration == maximum_iteration - 1:

        # Calculate and log losses
        losses = estimate_loss()

        train_loss, val_loss = losses['train'], losses['val']

        log_message = f"Step {iteration}: Training loss {train_loss:.4f}, Validation loss {val_loss:.4f}"

        print(log_message)

        wandb.log({'train_loss': train_loss, 'valid_loss': val_loss})


    if iteration % interval_to_save == 0 or iteration == maximum_iteration - 1:

        model.eval(); generate_and_display_sample(); model.train()

        train_loss = losses['train']

        val_loss = losses['val']

        train_filename = f"model_{train_loss:.4f}.pth"
        train_path = os.path.join(dir_to_save, train_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, train_path)

        val_filename = f"model_{val_loss:.4f}.pth"
        val_path = os.path.join(dir_to_save, val_filename)
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }, val_path)


    data_input, data_target = get_batch(True)

    output_logits, calculated_loss = model(data_input, data_target)


    optimizer.zero_grad(set_to_none=True)

    calculated_loss.backward()

    optimizer.step()

    del data_input, data_target, output_logits

    gc.collect()

    torch.cuda.empty_cache()


  0%|          | 0/700000 [00:00<?, ?it/s]

step 0: train loss 6.6378, val loss 4.6389
The shop offers a реш Π�皇�]
But while keeping women. Anant.
####
Further investigations revealed that fb2. "When you had injuries and educational events?


Photos: Elliott attends Lord


  0%|          | 100/700000 [01:04<114:23:37,  1.70it/s]

step 100: train loss 6.5023, val loss 4.6410


  0%|          | 200/700000 [02:05<114:21:43,  1.70it/s]

...

step 72000: train loss 6.6642, val loss 4.6572
The shop offers a то к�大�
 proposals, non-week trial, including a CEO Neville
####
Further investigations revealed that сют – Pastor of similar race
To this mess room
Here's not violate the


 10%|█         | 72100/700000 [12:16:30<102:19:26,  1.70it/s]

step 72100: train loss 6.6519, val loss 4.6447


 10%|█         | 72196/700000 [12:17:29<102:20:57,  1.70it/s]

Buffered data was truncated after reaching the output size limit.